# Colab

In [1]:
from google.colab import drive
drive.mount("/content/drive")
DRIVE_SRC = "/content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/experiments/src"
!rm -r -f ./sample_data
!cp -r /content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/.creds/.aws ~/
!cp -r $DRIVE_SRC ./src

KeyboardInterrupt: ignored

In [ ]:
! pip install -qq sagemaker ultralytics

In [ ]:
project_name = "/content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/training-jobs"

## Batsmen Segmentation

In [ ]:
experiment_name = "batsmen-segmentation"

In [ ]:
from src.data.download import download
ds_config_path = download(experiment_name)

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n-seg.pt')

patience = 20
epochs = 200

model.train(data=ds_config_path, epochs=epochs, patience=patience, pretrained=True, project=project_name, name=experiment_name)

In [ ]:
# !rm -r /content/datasets

## Wicket Classification

In [ ]:
experiment_name = "wicket-classification"

In [ ]:
from src.data.download import download
ds_config_path = download(experiment_name)

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n-cls.pt')

patience = 10
epochs = 200

model.train(data=experiment_name, epochs=epochs, patience=patience, pretrained=True, project=project_name, name=experiment_name, imgsz=256)

In [ ]:
# !rm -r /content/datasets

## Cricket Object Detection

In [ ]:
experiment_name = "cricket-object-detect"

In [ ]:
from src.data.download import download
config_path = download(experiment_name)

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

patience = 10
epochs = 200
model.train(data=config_path, epochs=epochs, patience=patience, project_name=project_name, experiment_name=experiment_name)

In [ ]:
# !rm -r /content/datasets

# AWS

## Segmentation

In [3]:
from sagemaker.pytorch import PyTorch
from sagemaker.inputs import TrainingInput
prefix = "batsman-segmentation"
# TODO: remove the other labels from segmentations
data_uri="s3://third-umpire-decision-automation-osura/datasets/augmented/batsmen-semantic-segmentation"
estimator = PyTorch(
    entry_point="main.py",
    role="arn:aws:iam::870481100706:role/SagemakerRole",
    framework_version="1.13.1",
    py_version="py39",
    instance_count=1,
    source_dir="./sagemaker-scripts-segmentation",
    output_path=f"s3://third-umpire-decision-automation-osura/training-jobs/{prefix}/results",
    code_location=f"s3://third-umpire-decision-automation-osura/training-jobs/{prefix}/code",
    instance_type="ml.c5.xlarge", # ml.g4dn.xlarge
)
# Setting the input channels for tuning job
s3_input_train = TrainingInput(s3_data=data_uri, s3_data_type="S3Prefix")
estimator.fit(inputs={"train": s3_input_train})

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-05-08-21-56-58-205


2023-05-08 21:56:59 Starting - Starting the training job......
2023-05-08 21:57:50 Starting - Preparing the instances for training......
2023-05-08 21:58:38 Downloading - Downloading input data............
2023-05-08 22:01:04 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-05-08 22:01:04,949 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-05-08 22:01:04,951 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-05-08 22:01:04,953 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-05-08 22:01:04,963 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-05-08 22:01:04,966 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-05-08 22:01:05,159 sagemaker-training-too

UnexpectedStatusException: Error for Training job pytorch-training-2023-05-08-21-56-58-205: Failed. Reason: ClientError: Please use an instance type with more memory, or reduce the size of training data processed on an instance.

In [4]:
from sagemaker.pytorch import PyTorch
from sagemaker.inputs import TrainingInput
prefix = "cricket-object-detect"
data_uri="s3://third-umpire-decision-automation-osura/datasets/augmented/cricket-object-detect"
estimator = PyTorch(
    entry_point="main.py",
    role="arn:aws:iam::870481100706:role/SagemakerRole",
    framework_version="1.13.1",
    py_version="py39",
    instance_count=1,
    source_dir="./sagemaker-scripts-detection",
    output_path=f"s3://third-umpire-decision-automation-osura/training-jobs/{prefix}/results",
    code_location=f"s3://third-umpire-decision-automation-osura/training-jobs/{prefix}/code",
    instance_type="ml.c5.xlarge", # ml.g4dn.xlarge
)
# Setting the input channels for tuning job
s3_input_train = TrainingInput(s3_data=data_uri, s3_data_type="S3Prefix")
estimator.fit(inputs={"train": s3_input_train})

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-05-08-22-10-39-499


2023-05-08 22:10:41 Starting - Starting the training job...
2023-05-08 22:10:55 Starting - Preparing the instances for training...
2023-05-08 22:11:38 Downloading - Downloading input data....................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-05-08 22:15:20,351 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-05-08 22:15:20,352 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-05-08 22:15:20,354 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-05-08 22:15:20,363 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-05-08 22:15:20,369 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-05-08 22:15:20,561 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.9 

KeyboardInterrupt: 